# <span>Title generation using GPT2 ✒️ </span>
<hr style="border-bottom: solid;background-color:light;color:black;">

## Introduction
   
<p style="text-align:justify; padding:20px;">
    In this notebook, we will focus on NLP. More spcifically on text generation. With the apparation of transformers in 2017, text generation have become one of the hot topics of datasience. To follow this trend, we are going to create new movie title from the Netflix dataset. To acheive this goal, we will deploy the distil version of the GPT2 transformer and fine tune the model. The notebook will be devided into 6 sections: 
</p>

* [Imports](#section-1)
* [Data preparation](#section-2)
* [Training the model](#section-3)
* [Generating titles](#section-4)

<a id="section-1"></a>
# <span>1. Imports</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [1]:
# import evaluate

# print(evaluate.list_evaluation_modules())
# metric = evaluate.load("glue", "mrpc")
# metric.compute(predictions=preds, references=predictions.label_ids)

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter("ignore")


In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

Embedding(50259, 768)

<a id="section-2"></a>
# <span>2. Data preparation</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [4]:
titles = pd.read_csv('../input/spotify-dataset/spotify_songs_en_pop.csv')['lyrics']
print(titles[0])
# Corpus = ''
# for listitem in titles:
#     Corpus += listitem

Get up out of my business You don't keep me from turning up 모두 다 여긴 Witness 넌 바른 척하는 Criminal 찔리는 걸 You feel it 너 네가 뭔데 왜 솔직하지 못해 넌 다를 건 대체 뭔데 aye Bad boy 그래 난 Bad boy 생각해 네가 하고픈 대로 That boy 그래 난 That boy 모르면 믿고 싶은 대로 Nothing ‘bout me nada you don't know Nothing ‘bout me not a thing 욕을 하는 이윤 다 있는 거야 나 아닌 너 안에 Cause we Party yeah we party we don't worry ‘bout the drama no Body Everybody 우린 아침까지 Wilding up 신경 쓸건 따로 아 몰라 오늘 다 집어치워 눈치 보지 말고 Volume up up up We go dumb dumb dumb dumb 누가 뭐래도 Litty dumb Litty dumb litty Bang the drum 갈 때까지 Baby stupid Stupid dumb dumb dumb dumb 누가 뭐래도 Litty dumb Litty dumb litty Bang the drum 갈 때까지 Baby we go 몰입하니 눈 몰리는 이치 시시콜콜 무료함을 완충해 내키면 곧장 Dumb litty 이 노래가 흐름 Come get it Drippin' 여기 주도권을 Grip than 거리낌 허물어 Let's go party 돌변함은 때 때에 따라 아무렴 어때 까무러치게 더불어 즐겨 사는 거지 My way follow nobody Party yeah we party we don't worry ‘bout the drama no Body Everybody 우린 아침까지 Wilding up 신경 쓸건 따로 아 몰라 오늘 다 집어치워 터지게 더 올려 Volume up up up We go dumb dumb dumb dumb 누가 뭐래도 Litty dumb

In [5]:
# max_length = max([len(tokenizer.encode(title)) for title in titles])
max_length = 1024

In [6]:
class NetflixDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>',
                                       max_length=max_length, padding="max_length")

            tensor = torch.tensor(encodings_dict['input_ids'])            
            if len(tensor) <= 1024:
                self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
                self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [7]:
#Keeping only a limited set of characters. 
to_remove = ['*', '_', '~', '¡', '«', '´', '¿', 'à',
             'á', 'è', 'é', 'ê', 'í', 'ï', 'ñ', 'ó', 'ú', 'ü', 'а', 'в', 'д', 'е', 'з',
             'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'ы', 'ь',
             'ю', 'я', '\u2005', '\u200a', '\u200b', '\u3000', '、', '「', '」', 'ぁ', 'あ',
             'い', 'う', 'え', 'か', 'が', 'き', 'ぎ', 'く', 'け', 'げ', 'こ', 'さ', 'し', 'じ',
             'す', 'せ', 'そ', 'た', 'だ', 'ち', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な',
             'に', 'ぬ', 'ね', 'の', 'は', 'び', 'ふ', 'ぶ', 'へ', 'べ', 'ほ', 'ま', 'み', 'む',
             'め', 'も', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん', 'イ', 'ウ', 'カ', 
             'キ', 'ギ', 'ク', 'シ', 'ス', 'タ', 'ダ', 'ッ', 'ツ', 'テ', 'ト', 'ネ', 'ノ', 'ビ', 
             'ポ', 'ミ', 'メ', 'モ', 'ヤ', 'ョ', 'リ', 'ル', 'レ', 'ン', 'ー', '一', '上', '世', 
             '中', '乗', '事', '人', '仆', '今', '他', '代', '体', '何', '侵', '俺', '像', '僕', 
             '償', '先', '光', '全', '共', '分', '切', '別', '前', '動', '去', '口', '叫', '名', 
             '向', '君', '呼', '咲', '嘘', '在', '壁', '壊', '声', '変', '夜', '夢', '大', '孤',
             '宿', '対', '尽', '巡', '巻', '序', '度', '弱', '張', '強', '影', '待', '後', '心',
             '恍', '恐', '悟', '悪', '情', '惚', '想', '愛', '感', '戻', '手', '抜', '拓', '指', 
             '振', '探', '揺', '支', '教', '日', '明', '昔', '昙', '時', '書', '最', '未', '来', 
             '極', '歓', '止', '正', '歩', '比', '気', '泣', '溺', '無', '特', '独', '珠', '現',
             '理', '由', '界', '痛', '白', '目', '真', '着', '瞳', '知', '破', '磨', '私', '突',
             '章', '笑', '筋', '続', '置', '肥', '胸', '自', '良', '薇', '薔', '蝕', '行', '衝', 
             '裸', '襲', '見', '覚', '见', '触', '言', '誇', '誰', '越', '踠', '身', '輝', '过',
             '返', '迷', '進', '過', '配', '重', '間', '闇', '限', '頭', '食', '高', '鳴', '갑',
             '개', '걱', '것', '겐', '겠', '견', '결', '곁', '곡', '곳', '과', '광', '교', '국',
             '군', '귀', '글', '급', '긋', '깊', '깐', '깜', '깝', '꺾', '께', '껴', '꼬', '꼭',
             '꼴', '꽃', '꽉', '꽤', '꿰', '끊', '끽', '남', '낮', '냥', '넓', '녁', '녔', '녕',
             '놀', '높', '놓', '놔', '놨', '뉴', '능', '닐', '님', '답', '당', '닿', '덜', '덩',
             '델', '독', '동', '된', '될', '둑', '둘', '둡', '뒤', '드', '득', '듭', '딱', '땀',
             '땡', '떠', '떤', '떨', '떻', '떼', '뚜', '뚫', '뜨', '뜻', '띄', '락', '럭', '럼',
             '럽', '럿', '렘', '력', '렵', '렸', '록', '론', '롭', '루', '릇', '릭', '릴', '림',
             '릿', '막', '많', '맘', '맛', '망', '맞', '맡', '맣', '매', '맨', '맴', '먼', '멀',
             '며', '명', '몇', '목', '몸', '문', '민', '밀', '밖', '발', '밝', '밟', '백', '벌',
             '베', '벼', '볍', '복', '본', '볼', '봉', '붉', '뷰', '빤', '빨', '뻑', '뻥', '뿌',
             '뿜', '쁘', '살', '삶', '석', '설', '섬', '섭', '섹', '셀', '셋', '소', '손', '솝', 
             '송', '숑', '순', '숴', '쉬', '쉽', '쉿', '슈', '스', '슨', '슴', '습', '식', '실',
             '싫', '십', '쌔', '쌤', '쏟', '쏴', '쑤', '쓰', '씩', '씹', '앉', '알', '았', '앞',
             '약', '양', '얗', '언', '엇', '엔', '열', '엽', '였', '영', '옆', '옇', '예', '옥',
             '외', '요', '용', '운', '울', '웃', '웅', '원', '윈', '유', '으', '익', '작', '잘', 
             '잠', '잡', '재', '잭', '적', '절', '젊', '점', '조', '존', '죽', '준', '줌', '줘', 
             '줬', '쥐', '증', '짓', '짝', '짧', '쩌', '찍', '착', '참', '창', '찾', '채', '챘', 
             '천', '철', '청', '쳐', '쳤', '초', '추', '축', '출', '춤', '춰', '췬', '칙', '카', 
             '커', '켜', '콘', '콤', '콱', '쿵', '크', '큰', '큼', '킬', '타', '탁', '탓', '탬', 
             '텐', '텝', '토', '통', '툭', '툰', '툴', '트', '특', '틀', '팅', '파', '팔', '패',
             '퍼', '펴', '편', '펼', '폈', '평', '포', '폼', '표', '품', '프', '플', '피', '필',
             '학', '할', '항', '핵', '행', '향', '험', '헤', '헷', '혀', '현', '호', '혹', '혼',
             '화', '확', '환', '황', '회', '효', '후', '훅', '훔', '흔', '흘', '히', '힘', 'ﬁ', 
             '\ufeff', '\xa0', 'É', 'ä', 'å', 'ç', 'ì', 'î', 'ò', 'ô', 'ö', 'ù', 'А', 'П',
             '가', '각', '간', '갈', '감', '같', '걍', '거', '건', '걷', '걸', '겁', '게', '겨', 
             '경', '계', '고', '곧', '골', '공', '관', '구', '굴', '궁', '그', '극', '근', '금', 
             '기', '긴', '길', '까', '깨', '꺼', '꾸', '꿈', '끄', '끈', '끌', '끔', '끝', '끼', 
             '낌', '나', '난', '날', '내', '냐', '너', '넌', '널', '넘', '네', '노', '논', '누', 
             '눈', '느', '는', '늘', '니', '닌', '다', '단', '달', '대', '더', '덕', '던', '데', 
             '도', '돌', '돼', '되', '두', '뒷', '든', '듣', '들', '듬', '듯', '디', '딜', '따', 
             '때', '떡', '또', '똑', '뛰', '라', '란', '람', '랑', '래', '랠', '러', '런', '렇', 
             '레', '려', '례', '로', '르', '른', '를', '름', '릎', '리', '린', '마', '만', '말', 
             '머', '먹', '멈', '멋', '멘', '면', '모', '몰', '못', '무', '묻', '물', '뭐', '뭔', 
             '뭘', '미', '믿', '바', '반', '받', '밤', '방', '배', '뱉', '버', '번', '범', '법', 
             '벗', '벽', '변', '별', '보', '봐', '부', '분', '불', '비', '빌', '빛', '빠', '뻔',
             '뿐', '뿔', '삐', '사', '상', '새', '생', '서', '선', '성', '세', '셔', '속', '쇠',
             '수', '숨', '슬', '시', '신', '심', '싶', '써', '썰', '아', '악', '안', '않', '애', 
             '앨', '야', '어', '억', '얼', '엄', '없', '었', '에', '여', '엮', '오', '온', '올', 
             '와', '완', '왔', '왜', '왠', '욕', '우', '움', '워', '위', '은', '을', '음', '의',
             '이', '인', '일', '임', '입', '있', '잊', '자', '잖', '장', '저', '전', '정', '제',
             '젠', '져', '좀', '좋', '주', '줄', '중', '즌', '즐', '지', '직', '진', '질', '짐', 
             '짜', '째', '쩜', '찌', '찔', '차', '처', '척', '체', '춘', '충', '취', '치', '친', 
             '칠', '침', '칵', '코', '콜', '키', '탈', '태', '터', '테', '투', '티', '팬', '펄', 
             '풀', '하', '한', '함', '해', '했', '허', '홀', '활', '흐', '힐', '々', 'お', 'ぐ', 
             'ば', 'ゃ', 'ア', 'ド', 'ブ', 'ム', '仲', '信', '値', '入', '受', '合', '味', '回', 
             '図', '場', '届', '平', '広', '意', '憧', '戦', '所', '拠', '描', '方', '望', '期',
             '消', '眺', '確', '空', '線', '証', '誓', '論', '起', '超', '逃', '違', '遠', '開',
             '集', '頂', '願', '駆', '鼓', '권', '넣', '둬', '뜰', '렴', '료', '밍', '션', '솔',
             '쓸', '윤', '응', '죄', '집', '텨', '픈', 'ŏ']

for i in range(len(titles)):
    for symbol in to_remove:
        titles[i] = titles[i].replace(symbol,"")

In [8]:
print(len(to_remove))

967


In [9]:
print(len(titles))
train_size = int(0.8 * len(titles))
test_size = len(titles) - train_size
dev_size = int(test_size/2)

titles, test_titles = train_test_split(titles, test_size=test_size)
val_dataset, test_dataset = train_test_split(test_titles, test_size=test_size-dev_size)

print(len(titles), len(test_titles))
train_dataset = NetflixDataset(titles, tokenizer, max_length=max_length)
print(len(train_dataset), len(val_dataset), len(test_dataset))

3739
2991 748
2601 374 374


In [10]:
# !pip install evaluate
# !pip install rouge_score

In [11]:
# import evaluate
# rouge = evaluate.load("rouge")

# sent = "We're c-coming out Got my flash on, it's true Need that picture of you, it's so magical We'd be so fantastical Leather and jeans Garage glamorous Not sure what it means But this photo of us, it don't have a price Ready for those flashing lights"
# ref = "don't you know what I'm talking about? It's just a matter of time before we get too close to each other So let me put your hands on my shoulder And make sure that no one else is hurtin' us 'Cause when it all comes crashing down (Oh-oh) You're not gonna be alone in this world for long Time will come and say I'll never go away But if there ever was somebody who could save our lives"
# # sent_list = sent.split()
# # ref_list = ref.split()[:len(sent_list)]
# print(len(sent), len(ref))
# ref = ref[:len(sent)]

# print([ref], [sent])
# bleu_result = rouge.compute(references=[ref], predictions=[sent])
# print(bleu_result)
#from datasets import load_metric

#rouge = load_metric("rouge")
#ref = ["We're c-coming out Got my flash on, it's true Need that picture of you, it's so magical We'd be so fantastical Leather and jeans Garage glamorous Not sure what it means But this photo of us, it don't have a price Ready for those flashing lights"]
#sent = ["don't you know what I'm talking about? It's just a matter of time before we get too close to each other So let me put your hands on my shoulder And make sure that no one else is hurtin' us 'Cause when it all comes crashing down (Oh-oh) You're not gonna be alone in this world for long Time will come and say I'll never go away But if there ever was somebody who could save our lives"]

# print([ref], [sent])
#bleu_result = rouge.compute(references=ref, predictions=sent)
#print(bleu_result)


<a id="section-3"></a>
# <span>3. Training the model</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [12]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=10, 
                                  per_device_train_batch_size=1, warmup_steps=10, weight_decay=0.05, 
                                  logging_dir='./logs', save_strategy = "no", report_to = 'none')

In [13]:
Trainer(model=model,  args=training_args, train_dataset=train_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

***** Running training *****
  Num examples = 2601
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 26010


Step,Training Loss
500,1.407000
1000,0.916900
1500,0.867200
2000,0.847700
2500,0.855600
3000,0.823300
3500,0.807100
4000,0.810000
4500,0.798100
5000,0.795300




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=26010, training_loss=0.723004010034405, metrics={'train_runtime': 3388.4378, 'train_samples_per_second': 7.676, 'train_steps_per_second': 7.676, 'total_flos': 6796328482897920.0, 'train_loss': 0.723004010034405, 'epoch': 10.0})

<a id="section-4"></a>
# <span>4. Generating titles</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [14]:
# results = []
# dict_results = {}
# count = 0

# for title in val_dataset:
#     beginning = " ".join(title.split()[0:5])
#     new_titles = {
#         'seed': beginning
#     }

#     generated = tokenizer('<|startoftext|>' + beginning, return_tensors="pt").input_ids.cuda()
#     sample_outputs = model.generate(generated,no_repeat_ngram_size = 1,num_beams=40, num_return_sequences=1,max_length=80,min_length=30)

#     new_titles['predictions'] = sample_outputs
#     results.append(new_titles)
    
#     dict_results[title] = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
#     count += 1
    

In [15]:
# import csv

# with open('/kaggle/working/genlyrics.csv', 'w+') as f:
#     writer = csv.writer(f)

#     header = ['Prompt', 'Generation', 'Coherence', 'Musical Quality', 'Pop Style']
#     writer.writerow(header)

#     for count, new_title in enumerate(results):
#         res1 = new_title['seed']
#         res2 = tokenizer.decode(new_title['predictions'][0], skip_special_tokens=True)
#         writer.writerow([res1, res2])

In [16]:
fake = []
real = []
dict_results = {}

count = 0

for title in test_dataset:
    print(title)
    beginning = " ".join(title.split()[0:5])
    new_titles = {
        'seed': beginning
    }
        
    if 0 <= count < 50:
        generated = tokenizer('<|startoftext|>' + beginning, return_tensors="pt").input_ids.cuda()
        sample_outputs = model.generate(generated,no_repeat_ngram_size = 1,num_beams=40, num_return_sequences=1,max_length=80,min_length=30)

        new_titles['predictions'] = sample_outputs
        fake.append(new_titles)
        dict_results[title] = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    
    elif 50 <= count < 100:
        new_titles['predictions'] = title
        real.append(new_titles)
        
    else:
        break
        
    count += 1

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I'm losing my self control Yeah, you're starting to trickle back in But I don't wanna fall down the rabbit hole Cross my heart, I won't do it again I tell myself, tell myself, tell myself, "Draw the line" And I do, I do But once in a while, I trip up, and I cross the line And I think of you Two years, and just like that My head still takes me back Thought it was done, but I Guess it's never really over Oh, we were such a mess But wasn't it the best? Thought it was done, but I Guess it's never really over Just because it's over doesn't mean it's really over And if I think it over, maybe you'll be coming over again And I'll have to get over you all over again Just because it's over doesn't mean it's really over And if I think it over, maybe you'll be coming over again And I'll have to get over you all over again I guess I could try hypnotherapy I gotta rewire this brain 'Cause I can't even go on the internet Without even checking your name I tell myself, tell myself, tell myself, "Draw t

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Friends are warmer than gold when you're old And keeping them is harder than you might suppose Lately, I tend to make strangers wherever I go Some of them were once people I was happy to know Mounting mileage on the dash Double darkness falling fast I keep stressing, pressing on Way deep down at some substratum Feels like something really wrong has happened And I confess I'm barely hanging on All my happiness is gone All my happiness is gone It's all gone somewhere beyond All my happiness is gone Ten thousand afternoons ago All my happiness just overflowed That was life at first and goal to go Me and you and us and them And all those people way back when All our hardships were just yardsticks then, you know You know It's not the purple hills It's not the silver lakes It's not the snowcloud shadowed interstates It's not the icy bike chain rain of Portland, Oregon Where nothing's wrong and no one's asking But the fear's so strong it leaves you gasping No way to last out here like this fo

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

So far, so drab Can't call it no towers on quest (So) (So) So worn unclear If I knew, then I wouldn't be stressed (So) (So) Get a child, fill your heart Bust down with the popcorn Cheap moto with some miles and a memoir Had a dream, re-emerge at the drug store Teenage love songs Score scenes worth Sundance, oh (Oh yeah) Surely I'll accept, oh (Oh) My choices breed regrets, oh (Oh) Hope soon soon there's none left, oh (Oh) Few folks seen me when my fingers fit the Cheerios Puffy Ami Yumi on the stereo 'Posed to be maestro for Juventus But events lead to venting for the check pen Please, unmarked barriers plenty Wish a nigga would try to press me Money on the raps like 'Bron We shouldn't be less than they see Oh, oh, ooh yeah Ooh (Ooh) Mhm, oh, mmm I love you, I love you, I love you, I love you I love you, I love you, I love you, I love you I love, yeah, you, yeah, you I get by, I get through I get over, I get moved Try my luck, don't try my trust I get by, I get through I get over, I ge

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I just called me a taxi Cause I ran into my ex here And I ain't down for that I'm way too old for that I passed a round in rotation I got a forty as my chaser You know I'm down for that And it got me thinking Is this really real? You know, I'm about it everyday Is this really real? You know, it gets cloudy everyday I've been touched by an angel I've been loved by a devil Got my panties around my ankles I've been touched by an angel That's the only thing I know Got my panties around my ankles I've been touched by an angel I've been loved by a devil Got my panties around my ankles I've been touched by an angel That's the only thing I know Got my panties around my ankles I've been touched by an angel


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Don't you know that my time's like a million dollars I could spend it anywhere Ridin' throughthenight in an'85, next to you Dancing through thelights and they flicker high, red and blue Everything is wrong, everything is right next to you Don't you know that my time's like a million dollars I could spend it anywhere I wanna waste, wanna waste my youth on you, you I wanna waste, wanna waste my youth on you, you Oh, the sounds running through the glass You can take the moment that I have, 'cause I wanna waste, wanna waste my youth on you, you Scared of growing up, looking out for love, breaks our hearts Staring at the sky, could we freeze the time? Crash this car No, I won't regret every seconds spend with you, ah Don't you know that my time's like a million dollars I could spend it anywhere I wanna waste, wanna waste my youth on you, you I wanna waste, wanna waste my youth on you, you Oh, the sounds running through the glass You can take the moment that I have, 'cause I wanna waste, wan

In [17]:
import csv # move up

with open('/kaggle/working/genlyrics.csv', 'w+') as f:
    writer = csv.writer(f)
    
    header = ['Prompt', 'Generation', 'Coherence', 'Musical Quality', 'Pop Style']
    writer.writerow(header)

    for new_title in fake:
        res1 = new_title['seed']
        res2 = tokenizer.decode(new_title['predictions'][0], skip_special_tokens=True)
        writer.writerow([res1, res2])
    
    for new_title in real:
        res1 = new_title['seed']
        res2 = new_title['predictions']
        writer.writerow([res1, res2])
        
with open('/kaggle/working/rougefile.txt', 'w+') as f:
    for res1,res2 in dict_results.items():
        f.write(res1)
        f.write('\n')
        f.write(res2)
        f.write('\n')

<a id="section-5"></a>
# <span>Work in progress</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

<p style="text-align:justify; padding:20px;">
    If you have any suggestions on how I could improve this notebook, please let me know :D
</p>
